### Advanced in silico drug design workshop. Olomouc, 30 January - 1 February, 2017.
### QSAR tutorial.
Dr. Pavel Polishchuk  
   
Institute of Molecular and Translational Medicine, Faculty of Medicine and Dentistry, Palacký University and University Hospital in Olomouc, Hněvotínská 1333/5, 779 00 Olomouc, Czech Republic  
http://imtm.cz  
http://qsar4u.com  
pavel_polishchuk@ukr.net  
pavlo.polishchuk@upol.cz

In [1]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors

ImportError: No module named rdkit

In [ ]:
# from rdkit.Chem.Draw import IPythonConsole
# from rdkit.Chem import Draw
# IPythonConsole.ipython_useSVG=True

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score, matthews_corrcoef
from sklearn.externals import joblib


#### Reading molecules and activity from SDF

In [ ]:
fname = "data/logBB.sdf"

mols = []
y = []
for mol in Chem.SDMolSupplier(fname):
    if mol is not None:
        mols.append(mol)
        y.append(mol.GetIntProp("logBB_class"))

#### Calculate descriptors (fingerprints) and convert them into numpy array

In [ ]:
# generate binary Morgan fingerprint with radius 2
fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols]

In [ ]:
def rdkit_numpy_convert(fp):
    output = []
    for f in fp:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [ ]:
x = rdkit_numpy_convert(fp)

In [ ]:
x.shape

In [ ]:
# check wether the data set is balanced
sum(y) / len(y)

#### Set random seed to make all further calculations reproducible

In [ ]:
seed = 42

#### Split the whole set on training and test sets

In [ ]:
# randomly select 20% of compounds as test set
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.20, random_state=seed)

#### Create folds for cross-validation

In [ ]:
cv = StratifiedKFold(n_splits=5, random_state=seed)

In [ ]:
# print out ids of folds
for i, (train_index, test_index) in enumerate(cv.split(x_tr, y_tr)):
    print("\nFold_" + str(i+1))
    print("TRAIN:", train_index)
    print("TEST:", test_index)

#### Scale X

This step may be crucial for certain modeling approaches lke SVM.
In the case of binary fingerprints it may be less useful.

In [ ]:
# obtain scale object which can be further applied to scale any data to fit the training set
scale = StandardScaler().fit(x_tr)
x_tr = scale.transform(x_tr)

In [ ]:
# it is a good idea to save it for future use
joblib.dump(scale, "data/logBB_scale.pkl", compress=3)

#### Search for optimal tuning parameters and build the model

In [ ]:
# create grid search dictionary
param_grid = {"max_features": [x_tr.shape[1] // 10, x_tr.shape[1] // 7, x_tr.shape[1] // 5, x_tr.shape[1] // 3], 
              "n_estimators": [100, 250, 500]}

In [ ]:
# setup model building
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [ ]:
# run model building
m.fit(x_tr, y_tr)

In [ ]:
m.best_params_

In [ ]:
m.best_score_

In [ ]:
m.cv_results_

In [ ]:
m.cv_results_['mean_test_score']

In [ ]:
m.cv_results_['params']

#### Save model

In [ ]:
joblib.dump(m, "data/logBB_rf_morgan.pkl", compress=3)

#### Predict test set compounds

In [ ]:
# load scale if necessary
scale = joblib.load("data/logBB_scale.pkl")

In [ ]:
# scale descriptors of the test set compounds
x_ts = scale.transform(x_ts)

In [ ]:
# predict logBB class
pred_rf = m.predict(x_ts)

In [ ]:
pred_rf

#### calc statistics for test set preditions

In [ ]:
accuracy_score(y_ts, pred_rf)

In [ ]:
matthews_corrcoef(y_ts, pred_rf)

In [ ]:
cohen_kappa_score(y_ts, pred_rf)

#### applicability domain estimates

In [ ]:
# if the model includes several ones like RF models or consensus models (or for probabilistic models)
# we can calculate consistency of predictions amongs those models and use it for estimation of applicability domain
pred_prob = m.predict_proba(x_ts)

In [ ]:
# probablity
pred_prob

In [ ]:
# setup threshold
threshold = 0.8

In [ ]:
# calc maximum predicted probability for each row (compound) and compare to the threshold
da = np.amax(pred_prob, axis=1) > threshold

In [ ]:
da

In [ ]:
# calc statistics
accuracy_score(np.asarray(y_ts)[da], pred_rf[da])

In [ ]:
matthews_corrcoef(np.asarray(y_ts)[da], pred_rf[da])

In [ ]:
cohen_kappa_score(np.asarray(y_ts)[da], pred_rf[da])

In [ ]:
# calc coverage
sum(da) / len(da)

#### Build SVM model

In [ ]:
# create grid search dictionary
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [ ]:
# setup model building
svm = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid, n_jobs=2, cv=cv, verbose=1)

In [ ]:
# run model building
svm.fit(x_tr, y_tr)

In [ ]:
svm.best_score_

In [ ]:
svm.best_params_

In [ ]:
# save model
joblib.dump(svm, "data/logBB_svm_morgan.pkl", compress=3)

In [ ]:
# predict logBB for the test set compounds
pred_svm = svm.predict(x_ts)

In [ ]:
pred_svm

In [ ]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred_svm))
print("MCC = ", matthews_corrcoef(y_ts, pred_svm))
print("Kappa = ", cohen_kappa_score(y_ts, pred_svm))

In [ ]:
# estimate applicability domain and calc stat
pred_prob = svm.predict_proba(x_ts)

In [ ]:
da = np.amax(pred_prob, axis=1) > threshold

In [ ]:
print("Accuracy = ", accuracy_score(np.asarray(y_ts)[da], pred_svm[da]))
print("MCC = ", matthews_corrcoef(np.asarray(y_ts)[da], pred_svm[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(y_ts)[da], pred_svm[da]))
print("Coverage = ", sum(da) / len(da))

### build the third model (GBM) compute consensus predictions from RF, and SVM models

In [ ]:
# setup model building
param_grid = {"n_estimators": [100, 200, 300, 400, 500]}
gbm = GridSearchCV(GradientBoostingClassifier(subsample=0.5, max_features=0.5), 
                   param_grid, n_jobs=2, cv=cv, verbose=1)

In [ ]:
# run model building
gbm.fit(x_tr, y_tr)

In [ ]:
gbm.best_score_

In [ ]:
gbm.best_params_

In [ ]:
pred_gbm = gbm.predict(x_ts)

In [ ]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred_gbm))
print("MCC = ", matthews_corrcoef(y_ts, pred_gbm))
print("Kappa = ", cohen_kappa_score(y_ts, pred_gbm))

#### consensus model

In [ ]:
pred_c = 1 * (((pred_rf + pred_svm + pred_gbm) / 3) >= 0.5)

In [ ]:
pred_c

In [ ]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred_c))
print("MCC = ", matthews_corrcoef(y_ts, pred_c))
print("Kappa = ", cohen_kappa_score(y_ts, pred_c))

### Add to Morgan fingerprints some other descriptors and look at the model performance

In [ ]:
# calc some descriptors
descr = []
for m in mols:
    descr.append([Descriptors.MolLogP(m),
                  Descriptors.TPSA(m),
                  Descriptors.NHOHCount(m),
                  Descriptors.NOCount(m),
                  Descriptors.NumHAcceptors(m),
                  Descriptors.NumHDonors(m),
                  Descriptors.NumRotatableBonds(m),
                  Descriptors.NumHeteroatoms(m),
                  Descriptors.FractionCSP3(m)])
descr = np.asarray(descr)

In [ ]:
descr.shape

In [ ]:
# add them to morgan fingerprints
x = np.concatenate((x, descr), axis=1)

In [ ]:
x.shape

In [ ]:
# randomly select 20% of compounds as test set
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.20, random_state=seed)

In [ ]:
scale = StandardScaler().fit(x_tr)
x_tr = scale.transform(x_tr)

In [ ]:
# create grid search dictionary
param_grid = {"max_features": [x_tr.shape[1] // 10, x_tr.shape[1] // 7, x_tr.shape[1] // 5, x_tr.shape[1] // 3], 
              "n_estimators": [100, 250, 500]}

In [ ]:
# setup model building
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [ ]:
# run model building
m.fit(x_tr, y_tr)

In [ ]:
m.best_score_

In [ ]:
# scale descriptors of the test set compounds
x_ts = scale.transform(x_ts)

In [ ]:
# predict logBB for the test set compounds
pred = m.predict(x_ts)

In [ ]:
pred

In [ ]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred))
print("MCC = ", matthews_corrcoef(y_ts, pred))
print("Kappa = ", cohen_kappa_score(y_ts, pred))

In [ ]:
# estimate applicability domain and calc stat
pred_prob = m.predict_proba(x_ts)

In [ ]:
da = np.amax(pred_prob, axis=1) > threshold

In [ ]:
print("Accuracy = ", accuracy_score(np.asarray(y_ts)[da], pred[da]))
print("MCC = ", matthews_corrcoef(np.asarray(y_ts)[da], pred[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(y_ts)[da], pred[da]))
print("Coverage = ", sum(da) / len(da))

The model has a better accuracy. Added descritors improved the model predictivity.

#### Let's try to analyse which variables are the most important in the model

In [ ]:
# rebuild RF model manually using best parameters to be able to extract additional information from the model
rf = RandomForestClassifier(n_estimators=m.best_params_["n_estimators"], 
                           max_features=m.best_params_["max_features"],
                           random_state=seed)
rf.fit(x_tr, y_tr)

In [ ]:
imp = rf.feature_importances_

In [ ]:
imp

In [ ]:
indices = np.argsort(imp)[::-1]

print("Feature ranking:")

# print top 10 features
for i in range(10):
    print("%d. feature %d (%f)" % (i + 1, indices[i], imp[indices[i]]))

2049 - MolLogP  
2050 - TPSA(m)  
2051 - NHOHCount  
2052 - NOCount 
2053 - NumHAcceptors  
2054 - NumHDonors  
2055 - NumRotatableBonds  
2056 - NumHeteroatoms  
2057 - FractionCSP3

features with numbers 1-2048 are different Morgan fingerprints  